In [1]:
import pyLDAvis.gensim
import spacy
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)


/opt/conda/lib/python3.7/site-packages/past/types/oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:627: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:637: D

In [2]:
#data module

import pandas as pd
from spacy.cli import download

def download_spacy_data():
    download("en_core_web_sm")

def validate_data(df):
    # check is as expected and described
    for category in df.category.unique():
        assert category in ['business', 'entertainment', 'politics', 'sport', 'tech']
    assert not any(df.duplicated())


def remove_duplicates(df):
    if any(df.duplicated()):
        dup = df.duplicated().value_counts()
        df = df[~df.duplicated()]
        print(f'{dup[True]} duplicated articles removed from dataset')
    return df

In [3]:
# read the data
df = pd.read_csv('../input/newsgroup20bbcnews/bbc-text.csv')
df = remove_duplicates(df)
validate_data(df)

99 duplicated articles removed from dataset


In [4]:
# summarise
df.describe()

,category,text
count,2126,2126
unique,5,2126
top,sport,tv future in the hands of viewers with home th...
freq,504,1


In [5]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("merge_noun_chunks")

# custom stop words
add_stop_words = ['say', r'\s', 'mr', 'Mr', 'said', 'says', 'saying', 'today', 'be', 'I']
for stopword in add_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

articles, article = [], []
doc_bin = DocBin()
dictionary = Dictionary()
corpus = []

# clean data
print('Processing articles using spaCy, removing punctuation, stop words, numbers and lemminizing')
for doc in tqdm(nlp.pipe(df.text.values, disable=["tok2vec"])):
    doc_bin.add(doc)
    article = [
        token.lemma_ for token in doc 
        if not token.is_stop 
        and not token.is_punct 
        and not token.like_num
        and not token.is_space]
    dictionary.add_documents([article])
    corpus.append(dictionary.doc2bow(article))

print('Building LDA model')
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
lda_model.show_topics()

Processing articles using spaCy, removing punctuation, stop words, numbers and lemminizing


2126it [02:05, 16.99it/s]


Building LDA model


[(0,
  '0.023*"s" + 0.006*"people" + 0.004*"uk" + 0.004*"new" + 0.004*"year" + 0.003*"mobile" + 0.003*"government" + 0.003*"bank" + 0.003*"blair" + 0.003*"told"'),
 (1,
  '0.018*"s" + 0.005*"people" + 0.005*"labour" + 0.004*"e" + 0.004*"new" + 0.003*"blair" + 0.003*"brown" + 0.003*"think" + 0.003*"companies" + 0.003*"uk"'),
 (2,
  '0.021*"s" + 0.005*"m" + 0.005*"t" + 0.005*"$" + 0.004*"£" + 0.004*"p2p" + 0.004*"government" + 0.004*"year" + 0.004*"new" + 0.003*"told"'),
 (3,
  '0.015*"s" + 0.008*"labour" + 0.008*"year" + 0.007*"election" + 0.006*"sales" + 0.006*"party" + 0.005*"blair" + 0.004*"minister" + 0.004*"people" + 0.004*"tax"'),
 (4,
  '0.020*"s" + 0.011*"$" + 0.009*"dollar" + 0.006*"year" + 0.005*"market" + 0.005*"growth" + 0.005*"prices" + 0.004*"bank" + 0.004*"new" + 0.004*"£"'),
 (5,
  '0.016*"s" + 0.003*"people" + 0.003*"year" + 0.003*"gm" + 0.003*"nielsen" + 0.003*"new" + 0.003*"bank" + 0.002*"real" + 0.002*"$" + 0.002*"hogan"'),
 (6,
  '0.023*"s" + 0.005*"players" + 0.005

In [6]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.034750  0.057374       1        1  19.294735
0      0.030476  0.016266       2        1  17.765084
6     -0.002725 -0.021480       3        1  15.493520
4      0.093572  0.009191       4        1  11.218688
3      0.063740 -0.074567       5        1   8.583432
2      0.051515 -0.012209       6        1   8.432682
1     -0.020294 -0.037228       7        1   6.932019
5     -0.031470 -0.001894       8        1   4.996577
9     -0.011740  0.111188       9        1   4.534906
7     -0.138324 -0.046641      10        1   2.748357, topic_info=          Term         Freq        Total Category  logprob  loglift
2031         o   402.000000   402.000000  Default  30.0000  30.0000
1230    dollar   504.000000   504.000000  Default  29.0000  29.0000
187          s  8416.000000  8416.000000  Default  28.0000  28.0000
762     labour   719.000000   719.000000  Default  27.0000  27.0000
142   networks   336.000000   336.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
152     people    31.881374  1781.772975  Topic10  -5.9000  -0.4292
840       year    31.017504  1995.530085  Topic10  -5.9274  -0.5699
1820  messages    24.100085   155.543466  Topic10  -6.1798   1.7295
998      world    27.978878  1098.849089  Topic10  -6.0305  -0.0764
1804         e    23.477831   235.583254  Topic10  -6.2059   1.2882

[823 rows x 6 columns], token_table=      Topic      Freq Term
term                      
245       1  0.082166    $
245       2  0.121523    $
245       3  0.182285    $
245       4  0.372166    $
245       5  0.033143    $
...     ...       ...  ...
354       6  0.154667    £
354       7  0.044616    £
354       8  0.022804    £
354       9  0.021812    £
354      10  0.015863    £

[3561 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 1, 7, 5, 4, 3, 2, 6, 10, 8])